##Example segmentation of gastrointestinal polyp images using u-nets and variants in Pytorch

Data used in this notebook is obtained from the   [**Kvasir SEG**](https://datasets.simula.no/kvasir-seg/) dataset. It is an open-access dataset of gastrointestinal polyp images and corresponding segmentation masks, manually annotated and verified by an experienced gastroenterologist.

####**Description from the original source**

*The human gastrointestinal (GI) tract is made up of different sections, one of them being the large bowel. Several types of anomalies and diseases can affect the large bowel, such as colorectal cancer. Colorectal cancer is the second most common cancer type among women and third most common among men. Polyps are precursors to colorectal cancer, and is found in nearly half of the individuals at age 50 having a screening colonoscopy, and are increasing with age. Colonoscopy is the gold standard for detection and assessment of these polyps with subsequent biopsy and removal of the polyps. Early disease detection has a huge impact on survival from colorectal cancer, and polyp detection is therefore important. In addition, several studies have shown that polyps are often overlooked during colonoscopies, with polyp miss rates of 14%-30% depending on the type and size of the polyps. Increasing the detection of polyps has been shown to decrease risk of colorectal cancer. Thus, automatic detection of more polyps at an early stage can play a crucial role in improving both prevention of and survival from colorectal cancer. This is the main motivation behind the development of a Kvasir-SEG dataset.*

**Kvasir-SEG Dataset Details**

The Kvasir-SEG dataset (size 46.2 MB) contains 1000 polyp images and their corresponding ground truth from the Kvasir Dataset v2. The resolution of the images contained in Kvasir-SEG varies from 332x487 to 1920x1072 pixels. The images and its corresponding masks are stored in two separate folders with the same filename. The image files are encoded using JPEG compression, and online browsing is facilitated. The open-access dataset can be easily downloaded for research and educational purposes.

The bounding box (coordinate points) for the corresponding images are stored in a JSON file. This dataset is designed to push the state of the art solution for the polyp detection task.


###Modifications in the notebook version
For the purpose of this notebook all images have been downsampled to 512x512 pixels and bounding boxes adjusted accordingly. Images with superimposed ground truth segmentation masks and bounding boxes have been created for illustration purposes. The modified version of the data can be imported directly into the notebook by running the cell below. The modified dataset has been split into predefined training-/validation-/test splits with 80%, 10% and 10% of the data in the respective splits. The resulting size of the dataset i somewhat larger, so downloading may take a few minutes.

In [ ]:
#Get the data
import os

if os.path.isdir('kvasir_polyps') == False:

  !curl -X GET -u 'jkha:szZbB-wJDwQ-DNxD3-KgggK-XRq7M' 'https://nextcloud.sdu.dk/remote.php/webdav/Introduction_to_medical_robotics/kvasir_polyps.zip' -o 'kvasir_polyps.zip'

  !unzip 'kvasir_polyps.zip'
  os.remove('kvasir_polyps.zip')


In [ ]:
#print tree to get an idea about the structure of the data
for root_,dir_,file_ in os.walk('kvasir_polyps'):
  print(root_,dir_,file_)

In [ ]:
#explore the data

import matplotlib.pyplot as plt
import numpy as np
import random
from PIL import Image

train_data = 'kvasir_polyps/train'
val_data = 'kvasir_polyps/val'
test_data = 'kvasir_polyps/test'

print('Train data')
print('Images: ', len(os.listdir(f'{train_data}/images')))
print('Masks: ',len(os.listdir(f'{train_data}/masks')))
print('\n')
print('Val data')
print('Images: ',len(os.listdir(f'{val_data}/images')))
print('Masks: ', len(os.listdir(f'{val_data}/masks')))
print('\n')
print('Test data')
print('Images: ',len(os.listdir(f'{test_data}/images')))
print('Masks: ',len(os.listdir(f'{test_data}/masks')))
print('\n')

#plot some example images

sample_files = random.choices(os.listdir(f'{val_data}/images'),k=3)

fig = plt.figure(figsize=(10, 10))

columns = 1
rows = 3

print('Random example images from validation set')
for i in range(1, (columns*rows)+1):
    fig.add_subplot(rows, columns, i)
    im = Image.open(f'{val_data}/images/{sample_files[i-1].split(".")[0]}.jpg')
    msk = Image.open(f'{val_data}/masks/{sample_files[i-1].split(".")[0]}.png')
    ovl = Image.open(f'{val_data}/overlays/{sample_files[i-1].split(".")[0]}.png')
    #the label file is similar to the mask except the values for background/foreground are [0,1] rather than [0,255]
    #we use the labels rather than mask to train the models later
    lbl = Image.open(f'{val_data}/labels/{sample_files[i-1].split(".")[0]}.png')
    im = np.concatenate([im,msk,ovl,np.array(lbl)*50],axis=1)
    plt.axis('off')
    plt.imshow(im)


###Bounding box annotations
As mentioned in the describtion the images are also annotated with bounding boxes for individual polyps. The bounding boxes will not be used in this notebook, but to get an idea of how these annotations are stored an example of obtaining box coordinates and superimposing them onto the images is given below.  

In [ ]:
import json
import cv2

bb = f'{val_data}/val_bboxes_resized.json'


print('Sample from validation set with bounding box annotations\n')
fig = plt.figure(figsize=(10, 10))
with open(bb, 'r') as j:
    bb_kvasir = json.loads(j.read())

    for i, (k,content) in enumerate(list(bb_kvasir.items())[0:3]):
      fig.add_subplot(3, 1, i+1)
      print(k,content)
      im = np.array(Image.open(f'{val_data}/images/{k}.jpg'))

      for i,box in enumerate(bb_kvasir[k]['bbox']):
        #print json file line with information on bounding box
        print((bb_kvasir[k]['bbox'][i]['xmin'],bb_kvasir[k]['bbox'][i]['ymin']), (bb_kvasir[k]['bbox'][i]['xmax'],bb_kvasir[k]['bbox'][i]['ymax']))

        cv2.rectangle(im, (bb_kvasir[k]['bbox'][i]['xmin'],bb_kvasir[k]['bbox'][i]['ymin']), (bb_kvasir[k]['bbox'][i]['xmax'],bb_kvasir[k]['bbox'][i]['ymax']), [255,255,255], thickness=3)
      plt.axis('off')
      plt.imshow(im)



#Training a segmentation model in Pytorch
In the following examples of defining and training segmentation models in Pytorch will be given. Including how to create a dataset and dataloaders for efficient training and setting up a training loop for model parameter updates using a training set and running validation on a seperate validation set to monitor the model's ability to generalize outside the training data.   

In [ ]:
import cv2
import tqdm


train_images = sorted([os.path.join('kvasir_polyps/train/images',image)for image in os.listdir('kvasir_polyps/train/images')])

train_labels = sorted([os.path.join('kvasir_polyps/train/labels',label)for label in os.listdir('kvasir_polyps/train/labels')])

print(f'number of train images: {len(train_images)} and labels: {len(train_images)}')

#check if images and labels match
print(f'train image 0-4: {train_images[0:4]}')
print(f'train label 0-4: {train_labels[0:4]}')

val_images = sorted([os.path.join('kvasir_polyps/val/images',image)for image in os.listdir('kvasir_polyps/val/images')])

val_labels = sorted([os.path.join('kvasir_polyps/val/labels',label)for label in os.listdir('kvasir_polyps/val/labels')])

print(f'number of val images: {len(val_images)} and labels: {len(val_images)}')

#check if images and labels match
print(f'val image 0-4: {val_images[0:4]}')
print(f'val label 0-4: {val_labels[0:4]}')



Create a dataset class to fetch data from the downloaded archive

In [ ]:
from torch.utils.data import Dataset
from torchvision import transforms
from torch.utils.data import DataLoader

class Dataset(Dataset):
  def __init__(self, imagePaths, maskPaths, transforms):
    # store the image and mask filepaths, and augmentation
    # transforms
    self.imagePaths = imagePaths
    self.maskPaths = maskPaths
    self.transforms = transforms
  def __len__(self):
    # return the number of total samples contained in the dataset
    return len(self.imagePaths)
  def __getitem__(self, idx):
    # grab the image path from the current index
    imagePath = self.imagePaths[idx]
    # load the image from disk, swap its channels from BGR to RGB,
    # and read the associated mask from disk in grayscale mode
    image = cv2.imread(imagePath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    mask = cv2.imread(self.maskPaths[idx], 0)

    # check to see if we are applying any transformations
    if self.transforms is not None:
      # apply the transformations to both image and its mask
      image = self.transforms(image)

    mask = torch.from_numpy(mask[np.newaxis,:]).to(torch.float32)

    # return a tuple of the image and its mask
    return (image, mask)




##The Unet segmentation model

Below we define a segmentation model class using the torch nn api. The model is based on the [Unet architecture](https://lmb.informatik.uni-freiburg.de/people/ronneber/u-net/) which is a seminal architecture for biomedical image segmentation. Despite it beeing somewhat old (~2015) it is still a good baseline for segmentation and many of the more recent SOTA segmentation models still use the encoder-decoder structure with skip-connections introduced by this work.

The below model doesn't follow the original configuration to a key, but is fundamentally the same architecture.

**note:**
Even this rather basic architecture has a relative large number of parameters, and the resolution of the images used for training is relatively high. Training this model on this data will take a very long time on a CPU. Colab provides some free GPU compute but not a lot. If running this notebook and future deep learning work in Colab it may be a good idea to invest in some compute unit in [colab pro](https://colab.research.google.com/signup).

To enable GPU compute in this notebook go to Runtime --> change runtime type and choose accelarator for the notebook.

In [ ]:
import torch
import torch.nn as nn
from torchvision import models
from torch.nn.functional import relu
from torchsummary import summary

#Example u-net encoder-decoder model implementation for n-class segmentation

class encoder_block(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(encoder_block, self).__init__()

        self.block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

    def forward(self,x):
      x = self.block(x)
      return x

class decoder_block(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(decoder_block, self).__init__()
        self.block = nn.Sequential(
            nn.UpsamplingBilinear2d(scale_factor=2),
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
        )


    def forward(self,x):
        x = self.block(x)
        return x

class UNet(nn.Module):
    def __init__(self, in_channels=3, n_class=1,filters=[64,128,256,512]):
        super(UNet, self).__init__()


        self.e1 = encoder_block(in_channels,filters[0])
        self.e2 = encoder_block(filters[0],filters[1])
        self.e3 = encoder_block(filters[1],filters[2])
        self.e4 = encoder_block(filters[2],filters[3])



        self.d1 = decoder_block(filters[3],filters[2])
        self.d2 = decoder_block(filters[2]+filters[2],filters[1])
        self.d3 = decoder_block(filters[1]+filters[1],filters[1])

        self.seg_block = nn.Sequential(nn.UpsamplingBilinear2d(scale_factor=2),
            nn.Conv2d(filters[1]+filters[0],filters[0],kernel_size=3,padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(filters[0],n_class,kernel_size=1))
        # the upsampling layer 'nn.UpsamplingBilinear2d' could be replaced with
        # a transpose convolution 'nn.ConvTranspose2d'. This will add more
        # parameters to the model but may lead to different/better results


    def forward(self, x):
        # Encode
        x1 = self.e1(x)
        x2 = self.e2(x1)
        x3 = self.e3(x2)
        x4 = self.e4(x3)

        #decode
        x5 = self.d1(x4)

        x5 = torch.cat([x5,x3],1)

        x6 = self.d2(x5)

        x6 = torch.cat([x6,x2],1)

        x7 = self.d3(x6)

        x7 = torch.cat([x7,x1],1)

        x7 = self.seg_block(x7)


        return x7

#print a summary of the model

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

Segmentation_model = UNet(n_class=1).to(DEVICE)

summary(Segmentation_model,(3,512,512),1)

#delete from memory
del Segmentation_model



###Configure som settings for this model and training setup

In [ ]:
# define the number of channels in the input, number of classes,

NUM_CHANNELS = 3
NUM_CLASSES = 1

# initialize learning rate, number of epochs to train for, and the
# batch size
INIT_LR = 0.0001
NUM_EPOCHS = 40
BATCH_SIZE = 4

# define threshold to filter weak predictions
THRESHOLD = 0.5

transformations = transforms.Compose([transforms.ToPILImage(),
                                 transforms.ToTensor()])

# create the train and test datasets
trainDS = Dataset(imagePaths=train_images, maskPaths=train_labels,
	transforms=transformations)
valDS = Dataset(imagePaths=val_images, maskPaths=val_labels,
    transforms=transformations)





In [ ]:
torch.cuda.empty_cache() #run this command to release GPU memory occupied by model/data

Define a function for model training. The function takes the model, training and validation data, loss function and optimizer, number of epoch, device on which to model i training (cpu/gpu) as well as a string name under which the best performing model is saved and can later be used for inference on test data.

After each iteration the loss function calculates a loss/cost quantifying the models performance/ability to solve the problem (segmenting polyps). In this example we use the [binary crossentropy loss](https://ml-cheatsheet.readthedocs.io/en/latest/loss_functions.html) with logits. This function takes the output of the last layer of the model at each pixel position and squeezes them between [0,1] signifying a probability of each pixel being a part of a polyp (p=1 if the model is absolutely confident in the pixel beeing a polyp) og background (p=0 if the model is absolutely confident in the pixel being background/normal pixel). The total loss is calculated as the mean across all pixels in the image/batch of images. The theoretical minimum loss is equal to zero if we have a perfect model.

The optimizer is an algorithm for minimizing the loss of the model by calculating gradient of the model parameters with respect to the loss. This gradient represents each parameters influence on the loss (positive or negative) and is used to update parameters by subtracting the gradient multiplied by a factor (learning rate/step size) thus (idealy) minimizing the loss of the model. This method of optimization is refered to as gradient decent and [diffenrent optimization algorithms exist](https://ml-cheatsheet.readthedocs.io/en/latest/optimizers.html#) for this purpose. The learning rate should be a small value (0.001 - 0.00001) to avoid noisy updates. If the learning rate is too high, minimizing the loss may fail due to noizy updates from large step sizes, or lead to overfitting (model does not generalize outside the training data). Consequently, if the learning rate is too low the parameter value updates may be too small to have an meaningfull effect on the loss. Learning the optimal set of parameteres may take exceptionally long or the model gets "stuck" in the "loss landscape" not reaching a (somewhat) optimal set of parameteres. The learning rate is a (crucial) hyperparameter (it can't be learned but found trough experiments/searching). To determine whether we have found a good learning rate we monitor the training loss during training ("if it gets lower = good") as well as the validation loss ("does it get lower as well = good. Does it rise/diverge signicantly from training loss = bad")      

In [ ]:
"""
Segmentation model performance is often measured using metrics describing the
similarity between the ground truth mask and the mask generated by the model.
One such metric is the Dice metric or dice score. To calculate the dice score
of our models we use the torch metrics library
"""
!pip install torchmetrics

In [ ]:
from torch.nn import BCEWithLogitsLoss
from torch.optim import Adam
from torch.nn.functional import sigmoid
import matplotlib.pyplot as plt
from torchmetrics.functional import dice

def train_function(model,trainDS,valDS,lossFunction=BCEWithLogitsLoss(),
                   optimizer = Adam,learningRate = 0.001,
                   epochs=40,batchSize=4, metric = dice, device="cpu",modelSaveName='model'):

  model.to(device)
  #metric.to(device)

  # calculate steps per epoch for training and test set
  trainSteps = len(trainDS) // batchSize
  valSteps = len(valDS) // batchSize

  # create the training and test data loaders
  trainLoader = DataLoader(trainDS, shuffle=True,
	  batch_size=batchSize,
	  num_workers=os.cpu_count())
  valLoader = DataLoader(valDS, shuffle=False,
	  batch_size=batchSize,
	  num_workers=os.cpu_count())

  opt = optimizer(model.parameters(), lr=learningRate)

  bestValLoss = 1e10

  # initialize a dictionary to store training history
  H = {"train_loss": [], "val_loss": [], "train_dice": [], "val_dice": []}

  for e in range(epochs):
    print(f"EPOCH: {e + 1}/{NUM_EPOCHS}")
    # set the model in training mode
    model.train()
    # initialize the total training and validation loss
    totalTrainLoss = 0
    totalValLoss = 0
    totalTrainDice = 0
    totalValDice = 0
    # loop over the training set
    print('Train phase:')
    for (i, (x, y)) in tqdm.tqdm(enumerate(trainLoader)):
  	  # send the input to the device
      (x, y) = (x.to(device), y.to(device))
      # perform a forward pass and calculate the training loss
      pred = model(x)
      loss = lossFunc(pred, y)
      dice_score = metric(sigmoid(pred),y.int())
      #print(dice_score)
      totalTrainDice += dice_score
      # first, zero out any previously accumulated gradients, then
      # perform backpropagation, and then update model parameters
      opt.zero_grad()
      loss.backward()
      opt.step()
      # add the loss to the total training loss so far
      totalTrainLoss += loss
    # switch off autograd
    print('\n')
    with torch.no_grad():
      # set the model in evaluation mode
      model.eval()
      # loop over the validation set
      print('val phase:')
      for (i,(x, y)) in tqdm.tqdm(enumerate(valLoader)):
        # send the input to the device
        (x, y) = (x.to(device), y.to(device))
        # make the predictions and calculate the validation loss
        pred = model(x)
        totalValLoss += lossFunc(pred, y)
        dice_score = metric(sigmoid(pred),y.int())
        totalValDice += dice_score
    # calculate the average training and validation loss
    avgTrainLoss = totalTrainLoss / trainSteps
    avgValLoss = totalValLoss / valSteps

    avgTrainDice = totalTrainDice / trainSteps
    avgValDice = totalValDice / valSteps
    # update our training history
    H["train_loss"].append(avgTrainLoss.cpu().detach().numpy())
    H["val_loss"].append(avgValLoss.cpu().detach().numpy())
    H["train_dice"].append(avgTrainDice.cpu().detach().numpy())
    H["val_dice"].append(avgValDice.cpu().detach().numpy())
    # print the model training and validation information

    print(f"Train loss: {avgTrainLoss:.6f}, Val loss: {avgValLoss:.4f}")
    print(f"Train dice: {avgTrainDice:.3f}, Val dice: {avgValDice:.3f}")

    #save the model performing best on the validation set
    if avgValLoss < bestValLoss:

      torch.save({'epoch': e,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': opt.state_dict(),
            'loss': avgValLoss},
	          f'{modelSaveName}.pth')
      bestValLoss = avgValLoss







    fig, ax = plt.subplots(nrows=1, ncols=4, figsize=(10, 5))

    im = np.moveaxis(x[0].detach().cpu().numpy(),0,-1)
    mask = np.moveaxis(np.stack([np.squeeze(y[0].detach().cpu().numpy(),axis=0)]*3,axis=0),0,-1)
    out = np.moveaxis(np.stack([np.squeeze(sigmoid(pred[0]).detach().cpu().numpy(),axis=0)]*3,axis=0),0,-1)
    print(f'min polyp pixel probability: {np.min(out)}')
    print(f'max polyp pixel probability: {np.max(out)}')
    bin = out.copy()
    bin[bin >= THRESHOLD] = 1
    bin[bin < THRESHOLD] = 0

    ax[0].imshow(im)
    ax[0].set_title(f"Input Image")

    ax[1].imshow(mask)
    ax[1].set_title(f"Reference Mask")

    ax[2].imshow(out)
    ax[2].set_title(f"Output Probability Map")

    ax[3].imshow(bin)
    ax[3].set_title(f"Model segmentation")

    plt.show()
    plt.close()

  return H



##Train the model
Here we can use the training function to optimize the model. The function returns a "history" (per epoch validation and training loss). we could plot this to get an idea about the models ability to generalize (overfitting), if the learning rate is apropiate, or whether the model is capable of solving the problem. If it can't be (over)fittet to the training set, it unlikely that we are going to find an optimal solution using this model. It is a nice, simple way to minitor training and eyeball hyperparameter settings before performing a more extensive search.

In colab you can adjust runtime settings to e.g. train using a GPU. This will decrease training time significantly compared to running on the cpu. To do this, go to runtime at the top of the page click on runtime type and select a GPU under hardware accelerator. Either way, training may take a while. The intend of the notebook i to give an example of using deep learning models for biomedical image segmentation. So in this next part you can just run the code as is or do experiments with different parameters to observe the effects, and then move on when it makes sense. The task is not to achieve a fully optimized model.  

In [ ]:
# determine the device to be used for training and evaluation
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


# initialize our UNet model
unet = UNet(in_channels=NUM_CHANNELS,n_class=NUM_CLASSES)
# initialize loss function and optimizer
lossFunc = BCEWithLogitsLoss(reduction="mean")
"""
train the model. With the default parameters in this notebook
it may take some time (~10 epochs)
before the model begins to assign high probability for polyp
to any pixels in the chosen validation smaple
"""

history = train_function(unet,trainDS,valDS,lossFunction=lossFunc,optimizer=Adam,
               learningRate = INIT_LR,
               epochs=NUM_EPOCHS,
               batchSize=BATCH_SIZE,
               device=DEVICE,
               modelSaveName='unet')

In [ ]:
torch.cuda.empty_cache() #run this command to release GPU memory

##Make use of the model
Once the model is trained we can use it to perform segmentation of new images by reinitalizing it and loading the optimized parameters.   

In [ ]:
#redefine model and load best parameters to run inference on test data
#This is left as an exercise
unet = UNet(in_channels=NUM_CHANNELS,n_class=NUM_CLASSES)
checkpoint = torch.load('unet.pth')

unet.load_state_dict(checkpoint['model_state_dict'])


In [ ]:
#clean up before continuing
try:
  del unet
except:
  pass

torch.cuda.empty_cache()

##More advanced + pre-trained models
Depending on the data, it might be the case that a relative simple model such as Unet won't be able to solve the problem (e.g. polyp segmentation). In this case, we may need to increase the complexity of the model by adding more layers, or reconfigure layers to include components such a [residual connections](https://arxiv.org/pdf/1512.03385.pdf) or [batch normalization](https://arxiv.org/pdf/1502.03167.pdf). It may also be the case that we simply don't have enough data to obtain a good, generalised model (regardless of complexity). In this case we can make use of pretrained models. Pretrainng is a method were the model is first trained one a larger dataset and the parameters obtained in this setting are then used as initial values and finetuned on a downstream task. For encoder-decoder type models such a Unet, a common approach is the use a model trained for classification on the [ImageNet dataset](https://www.image-net.org/index.php) as the encoder "backbone". A caveat with increasing complexity and making use of pretrained models is that this may lead to problems with overfitting. The best way compat overfitting is to train on more (representative) data. But in the medical domain this may not always be possible or atleast very costly. An alternative solution is to make use of relurazation techniques such as data augmentation and dropout ([as in the AlexNet paper](https://proceedings.neurips.cc/paper_files/paper/2012/file/c399862d3b9d6b76c8436e924a68c45b-Paper.pdf)). It is also a good idea to decrease the learning rate compared to when training on randomly initialized weights.

Below we can try to solve the same polyp segmentation problem as above, but using a unet with a pretrained backbone. To illustrate the difference between a model with random initialized parameters (or weights), we plot the some of the "filters" from the first layer of the model and observe how these have been optimized to react to different features in the input images during the pretraining process.  

In [ ]:
#Install the Segmentation-models-pytorch library to access predefined segmentation models with pretrained weights
#https://segmentation-modelspytorch.readthedocs.io/en/latest/#
!pip install segmentation-models-pytorch

In [ ]:
import segmentation_models_pytorch as smp

model = smp.Unet(encoder_name="resnet18",encoder_weights='imagenet',encoder_depth=4,decoder_channels = [256, 128, 64,32]).to(DEVICE)

summary(model,(3,512,512),1)

In [ ]:
def plot_kernels(tensor, num_cols=6):
    if not tensor.ndim==4:
        raise Exception("assumes a 4D tensor")
    #if not tensor.shape[-1]==3:
    #    raise Exception("last dim needs to be 3 to plot")
    num_kernels = tensor.shape[0]
    num_rows = 1+ num_kernels // num_cols
    fig = plt.figure(figsize=(num_cols,num_rows))
    for i in range(tensor.shape[0]):
        ax1 = fig.add_subplot(num_rows,num_cols,i+1)
        ax1.imshow(np.moveaxis(tensor[i],0,-1)*255)
        ax1.axis('off')
        ax1.set_xticklabels([])
        ax1.set_yticklabels([])

    plt.subplots_adjust(wspace=0.1, hspace=0.1)
    plt.show()

In [ ]:
#Initialize unet model with resnet encoder without pretrained weights and plot the weights of the first convolution layer with 7x7 filters

model_random = smp.Unet(encoder_name="resnet18",encoder_weights=None,encoder_depth=4,decoder_channels = [256, 128, 64,32]).to(DEVICE)

mm = model_random.double()
body_model = [i for i in mm.children()][0]
body_mm = body_model.double()
body_layers =  [i for i in body_mm.children()][0]

tensor = body_layers.weight.data.cpu().numpy()
plot_kernels(tensor)


In [ ]:
#Initialize unet model with resnet encoder with imagenet pretrained weights and plot the weights of the first convolution layer with 7x7 filters
model_imagenet = smp.Unet(encoder_name="resnet18",encoder_weights='imagenet',encoder_depth=4,decoder_channels = [256, 128, 64,32]).to(DEVICE)

mm = model_imagenet.double()
body_model = [i for i in mm.children()][0]
body_mm = body_model.double()
body_layers =  [i for i in body_mm.children()][0]

tensor = body_layers.weight.data.cpu().numpy()
plot_kernels(tensor)

In [ ]:
from torch.nn import BCEWithLogitsLoss
from torch.optim import Adam

#Initialize unet model with resnet encoder with imagenet pretrained weights and plot the weights of the first convolution layer with 7x7 filters
model_imagenet = smp.Unet(encoder_name="resnet18",encoder_weights='imagenet',encoder_depth=4,decoder_channels = [256, 128, 64,32])

# determine the device to be used for training and evaluation
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# initialize loss function and optimizer
lossFunc = BCEWithLogitsLoss(reduction="mean")

hist = train_function(model_imagenet,trainDS,valDS,lossFunction=lossFunc,optimizer=Adam,
               learningRate = INIT_LR*0.1,
               epochs=NUM_EPOCHS,
               batchSize=BATCH_SIZE,
               device=DEVICE,
               modelSaveName='smp-unet')

In [ ]:
torch.cuda.empty_cache()

In [ ]:
#Run this cell to remove the data from the colab directory
import shutil
shutil.rmtree('kvasir_polyps')